# Code Pudding 2024
***

The purpose of this notebook will be to analyse data retrieved from the [Spotify Web API](https://developer.spotify.com/documentation/web-api) in order to train various machine learning models to predict the genre of any given song. Once the models have been trained, validated and tested, a function will be built that feeds the data from the API to the best preforming model, and it's genre will be predicted.

## Initialization

In [15]:
import pandas as pd
import numpy as np
import plotly.express as px
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import dotenv_values

## Spotify API

The following cell reads the `.env` file into a dictionary with the following keys:

- `Client_ID`
- `Client_secret`

Ensure you have followed the directions from the [Web API](https://developer.spotify.com/documentation/web-api) and have both of those values.

In [17]:
config = dotenv_values(".env")  # config = {"Client_ID": "foo", "Client_secret": "foo"}
client_credentials_manager = SpotifyClientCredentials(client_id=config['Client_ID'], client_secret=config['Client_secret'])
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## EDA

In [40]:
data = pd.read_csv('spotify_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5829 entries, 0 to 5828
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          5829 non-null   object 
 1   genre             5829 non-null   object 
 2   danceability      5829 non-null   float64
 3   energy            5829 non-null   float64
 4   key               5829 non-null   int64  
 5   loudness          5829 non-null   float64
 6   mode              5829 non-null   int64  
 7   speechiness       5829 non-null   float64
 8   acousticness      5829 non-null   float64
 9   instrumentalness  5829 non-null   float64
 10  liveness          5829 non-null   float64
 11  valence           5829 non-null   float64
 12  tempo             5829 non-null   float64
 13  type              5829 non-null   object 
 14  uri               5829 non-null   object 
 15  track_href        5829 non-null   object 
 16  analysis_url      5829 non-null   object 


In [6]:
print(data.duplicated().sum())
print(data['track_id'].duplicated().sum())

0
0


In [41]:
data['genre'].value_counts()

genre
other        1500
rock          900
hip hop       900
jazz          898
classical     856
pop           775
Name: count, dtype: int64

## Search Function

Created by Isaiah Montoya

In [39]:
def get_track_features(song_title, artist_name):
    # Search for the song using Spotipy's search function
    result = sp.search(q=f"track:{song_title} artist:{artist_name}", type='track', limit=1)
    
    if result['tracks']['items']:
        # Extract the track ID from the search result
        track = result['tracks']['items'][0]
        track_id = track['id']
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        
        print(f"Found track: {track_name} by {artist_name}")
        
        # Use the track ID to get the song's features
        features = sp.audio_features(track_id)
        return features[0]  # Return the features dictionary
    else:
        print(f"No results found for {song_title} by {artist_name}")
        return None

# Example usage
song_title = "Spybreak-Short One"
artist_name = "Propellerheads"
features = get_track_features(song_title, artist_name)

if features:
    print("Audio Features:")
    print(features)

Found track: Spybreak! - Short One by Propellerheads
Audio Features:
{'danceability': 0.552, 'energy': 0.929, 'key': 1, 'loudness': -8.626, 'mode': 1, 'speechiness': 0.0458, 'acousticness': 7.14e-06, 'instrumentalness': 0.846, 'liveness': 0.18, 'valence': 0.466, 'tempo': 127.834, 'type': 'audio_features', 'id': '6AyXbkn4cwrErFIMbRBRDs', 'uri': 'spotify:track:6AyXbkn4cwrErFIMbRBRDs', 'track_href': 'https://api.spotify.com/v1/tracks/6AyXbkn4cwrErFIMbRBRDs', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6AyXbkn4cwrErFIMbRBRDs', 'duration_ms': 244400, 'time_signature': 4}
